In [ ]:
#Installs

!pip install nltk
!pip install -U spacy
!python -m spacy download en_core_web_lg

In [ ]:
#Imports 

import re
from collections import Counter
import spacy
word2vec = spacy.load('en_core_web_lg')
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [ ]:
#User functions

def preprocess(input_sentence):
    input_sentence = input_sentence.lower()
    input_sentence = re.sub(r'[^\w\s]','',input_sentence)
    tokens = word_tokenize(input_sentence)
    input_sentence = [i for i in tokens if not i in stop_words]
    return(input_sentence)

def compare_overlap(user_message, possible_response):
    similar_words = 0
    for token in user_message:
        if token in possible_response:
              similar_words += 1
    return similar_words
  
def extract_nouns(tagged_message):
    message_nouns = list()
    for token in tagged_message:
        if token[1].startswith("N"):
            message_nouns.append(token[0])
    return message_nouns

def compute_similarity(tokens, category):
    output_list = list()
    for token in tokens:
        output_list.append([token.text, category.text, token.similarity(category)])
    return output_list

In [82]:
#Responses

response_a = "For a {}ache take a painkiller!"
response_b = "If your {} is blue, consult your MD."
response_c = "In case of dry {}, drink plenty of water."
response_d = "It is not killing you. Rest, and your {} will get better."

blank_spot = "body"


responses = [response_a, response_b, response_c, response_d]

In [86]:
#Main

class ChatBot:
  exit_commands = ("quit", "goodbye", "exit")
  
  def make_exit(self, user_message):
    for exit in self.exit_commands:
      if exit in user_message:
        #print("Goodbye and have a nice day!") #Uncomment when chatting directly
        return True
      

  def chat(self):
    user_message = input("Hello this is ChatBot MD! What can I help you with? ")
    while not self.make_exit(user_message) == True:
      user_message = self.respond(user_message)
  

  def find_intent_match(self, responses, user_message):
    bow_user_message = Counter(preprocess(user_message))
    processed_responses = [Counter(preprocess(response)) for response in responses]

    similarity_list = [compare_overlap(bow_user_message, processed_response) for processed_response in processed_responses]
    response_index = similarity_list.index(max(similarity_list))
    return responses[response_index]
    
  
  def find_entities(self, user_message):
    tagged_user_message = pos_tag(preprocess(user_message))
    message_nouns = extract_nouns(tagged_user_message)
    tokens = word2vec(" ".join(message_nouns))
    category = word2vec(blank_spot)
    word2vec_result = compute_similarity(tokens, category)
    word2vec_result.sort(key=lambda x: x[2])
    if len(word2vec_result) == 0:
      return blank_spot
    else:
      return word2vec_result[-1][0]
      
 
  def respond(self, user_message):
    entity = self.find_entities(user_message)
    best_response = self.find_intent_match(responses, user_message)
    #print(best_response.format(entity))                              #Uncomment when chatting directly 
    #input_message = input("Can I help you with anything else? ")
    #return input_message
    return best_response.format(entity)


In [ ]:
#Run to chat directly

cb = ChatBot() 
cb.chat()

In [88]:
#Run to chat via GUI

from tkinter import *

# GUI
root = Tk()
root.title("ChatBot MD")

BG_GRAY = "#957CA1"
BG_COLOR = "#d4c6de"
TEXT_COLOR = "#821FB3"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 15 bold"


# Send function
def send():
	cb = ChatBot()
	send = "You : " + e.get()
	txt.insert(END, "\n" + "You : ", "bold")
	txt.insert(END, e.get())

	user = e.get().lower()
	if cb.make_exit(user) == True:
		txt.insert(END, "\n" + "ChatBot MD : ","bold")
		txt.insert(END, "Goodbye and have a nice day!")
	else:
		response = cb.respond(user)
		txt.insert(END, "\n" + "ChatBot MD : ","bold")
		txt.insert(END, response + " Can I help you with anything else?")

	e.delete(0, END)


lable1 = Label(root, bg=BG_COLOR, fg=TEXT_COLOR, text="ChatBot MD", font=FONT_BOLD, pady=10, width=20, height=1).grid(
	row=0)

txt = Text(root, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT, width=90)
txt.grid(row=1, column=0, columnspan=2)
txt.tag_configure("bold", font=FONT_BOLD)
txt.insert('1.0', ' Hello this is ChatBot MD! What can I help you with?')
txt.insert('1.0', 'ChatBot MD: ',"bold")

scrollbar = Scrollbar(txt)
scrollbar.place(relheight=1, relx=0.974)

e = Entry(root, bg="#d4c6de", fg=TEXT_COLOR, font=FONT, width=84)
e.grid(row=2, column=0)

send = Button(root, text="Send", font=FONT, bg=BG_GRAY,
			command=send).grid(row=2, column=1)

root.mainloop()